In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'QuadruppedWalk-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.bipedalwalker_lstm_config
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
a2c_config = games_configurations.quadrupped_lstm_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

robot lib not fouund
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-30 21:43:03,239	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-30_21-43-03_4007/logs.
2019-06-30 21:43:03,350	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:10804 to respond...
2019-06-30 21:43:03,500	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:42037 to respond...
2019-06-30 21:43:03,510	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-30 21:43:03,557	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-30_21-43-03_4007/logs.
2019-06-30 21:43:03,560	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:10804',
 'object_store_address': '/tmp/ray/session_2019-06-30_21-43-03_4007/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-30_21-43-03_4007/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'walker', obs_space, False, action_space, a2c_config)
#agent.restore('nn/walkerBipedalWalker-v2')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
4
Instructions for updating:
Use keras.layers.dense instead.


2019-06-30 21:43:04,808	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=4113) 
(pid=4113) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=4113) For more information, please see:
(pid=4113)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=4113)   * https://github.com/tensorflow/addons
(pid=4113) If you depend on functionality not listed there, please file an issue.
(pid=4113) 
(pid=4113) robot lib not fouund
(pid=4113) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=4113) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=4117) 
(pid=4117) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=4117) For more information, please see:
(pid=4117)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=4117)   * https://github.com/tensorflow/addons
(pid=4117) If you depend on functionality not listed there

(pid=4138) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=4138)   result = entry_point.load(False)
(pid=4138) robot lib not fouund
(pid=4138) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=4138) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=4146) robot lib not fouund
(pid=4146) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=4146) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=4146) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=4146)   result = entry_point.load

Frames per seconds:  910.3879063107266
Frames per seconds:  935.5864611003822
Frames per seconds:  951.2993687709628
Frames per seconds:  966.4556434266817
Frames per seconds:  984.8703265598045
Frames per seconds:  982.6311014034401
Frames per seconds:  975.8683297074077
Frames per seconds:  988.9243192559825
Frames per seconds:  991.1067501056448
Frames per seconds:  957.6148622066446
Frames per seconds:  983.9870740457629
Frames per seconds:  986.8968729721929
Frames per seconds:  966.8444772106043
Frames per seconds:  984.0148031777849
Frames per seconds:  980.8359962928913
Frames per seconds:  964.1288139778417
Frames per seconds:  968.2279706170443
Frames per seconds:  975.1549948579765
Frames per seconds:  959.0513862305745
Frames per seconds:  972.7361267832475
Frames per seconds:  968.5148632647295
Frames per seconds:  966.51577826519
Frames per seconds:  970.1501052745114
Frames per seconds:  962.6242425930179
Frames per seconds:  956.974008655389
Frames per seconds:  964.679

Frames per seconds:  956.9612685637557
Frames per seconds:  945.881104262383
Frames per seconds:  956.5946698785032
Frames per seconds:  962.6844410125747
Frames per seconds:  949.4837203199795
Frames per seconds:  961.6719955903812
Frames per seconds:  946.9583627836136
Frames per seconds:  954.269437527759
Frames per seconds:  964.1270284632333
Frames per seconds:  956.5700091526295
Frames per seconds:  944.0466325221046
Frames per seconds:  955.462281754736
Frames per seconds:  956.6161358650259
Frames per seconds:  927.4104276382504
Frames per seconds:  961.4616154513767
Frames per seconds:  951.495522758556
Frames per seconds:  942.8062476151841
Frames per seconds:  954.0906829558621
Frames per seconds:  955.4701462803675
Frames per seconds:  944.814347736564
Frames per seconds:  943.8353885807575
Frames per seconds:  953.5241774889432
Frames per seconds:  952.89940899819
Frames per seconds:  953.4528427679357
Frames per seconds:  963.2213813002545
Frames per seconds:  947.3172979

Frames per seconds:  817.7847538913261
Frames per seconds:  831.778337422215
Frames per seconds:  820.5562651547938
Frames per seconds:  822.5432764475053
Frames per seconds:  830.7469818461543
Frames per seconds:  835.2020603493019
Frames per seconds:  827.3919676484387
Frames per seconds:  834.9428469979397
Frames per seconds:  840.8362314098773
Frames per seconds:  825.8676993136064
Frames per seconds:  832.6100047819706
Frames per seconds:  830.8317121110409
Frames per seconds:  828.0632274949933
Frames per seconds:  830.9208803494809
Frames per seconds:  835.6391824565411
Frames per seconds:  820.9952537702272
Frames per seconds:  845.3055115815117
Frames per seconds:  820.5101388220116
Frames per seconds:  828.9747499027346
Frames per seconds:  826.5964575886151
Frames per seconds:  833.9664188583088
Frames per seconds:  820.5141359710213
Frames per seconds:  832.2957018963455
Frames per seconds:  833.8787409733031
Frames per seconds:  816.7485299603042
Frames per seconds:  828.7

In [ ]:

ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()